In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bagsolver.bag import Bag
from bagsolver.utils import load_bag_data, parse_solution
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed

from experiment2 import run

In [3]:
runs = {
    "branch_bound": ("solve_branch_bound", {}),
    "greedy_simple": ("solve_greedy", {"redux": False}),
    "greedy_redux": ("solve_greedy", {"redux": True}),
    "dynamic_cost": ("solve_dynamic_cost", {}),
    "dynamic_weight": ("solve_dynamic_weight", {}),
    "solve_ftapas_03": ("solve_ftapas", {"epsilon": 0.3}),
    "solve_ftapas_05": ("solve_ftapas", {"epsilon": 0.5}),
    "solve_ftapas_07": ("solve_ftapas", {"epsilon": 0.7}),
}

In [4]:
df = run(runs, 5, ThreadPoolExecutor, 3)

100%|██████████| 432/432 [02:24<00:00,  2.98it/s]

Saving results into results2.csv
